# My first classification notebook
El objetivo principal de este libro es aplicar mis conocimientos basicos de machine learning en un problema practico presentado en las competencias de Kaggle, esto con el fin de recibir retroalimentación de la comunidad, aprender de sus aportes y orientar a quienes recien empiezan.

En este cuaderno se llevará a cabo un proceso de machine learning aplicado a los datos "Titanic: Machine Learning From Desaster" con el fin de predecir los sobrevivientes del hundimiento del titanic en 1912. La descripción general de la competencia se encuentra aquí: https://www.kaggle.com/c/titanic

Para este proceso de predicción seguiré la siguiente estructura:
1. Definir el problema
2. Analizar los datos
3. Preparar los datos
4. Hacer predicciones
5. Mejorar los resultados

Esta estructura es bien definida en el siguiente sitio: https://machinelearningmastery.com/process-for-working-through-machine-learning-problems/ 

Comenzamos.

# 1. Definir el problema

A partir de un conjunto muestral de entrenamiento que enumera a los pasajeros que sobrevivieron al desastre del Titanic y a los que no, se debe crear un modelo que pueda determinar con base a un conjunto de test si dichos pasajeros sobreviven o no.

La información detallada del problema se encuentra en aquí: https://www.kaggle.com/c/titanic

# 2. Analizar los datos

Antes de iniciar la exploración y procesamiento de datos es necesario importar las librerias que usaremos a lo largo de nuestro proyecto.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #Gives us Graphics
import seaborn as sns #Libreria para gráficar
from sklearn.model_selection import cross_val_score #libreria para obtener puntuación de algoritmos
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Obtenemos nuestro conjunto de entrenamiento que será almacenado en la variable "titanicTrain", al mimsmo tiempo que obtenemos nuestro conjunto de prueba que será almacenado en la variable "titanicTest"

In [ ]:
#firts i import the data
titanicTrain = pd.read_csv("../input/titanic/train.csv")
titanicTest = pd.read_csv("../input/titanic/test.csv")

Hacemos una inspección rapida del contenido de cada conjunto de datos.

In [ ]:
titanicTrain.shape

In [ ]:
titanicTest.shape

In [ ]:
titanicTrain.head()

In [ ]:
titanicTest.head()

In [ ]:
titanicTrain.info()

In [ ]:
titanicTest.info()

# Observaciones
Observando ambos conjuntos de datos podemos apreciar que el conjunto de entrenamiento cuenta con 11 columnas, mientras el de test cuenta con solo 10. Esto es porque el conjunto de test no contiene la columna "Survived" que es la que indica si el tripulante sobrevive o no, y es con base a este conjunto de datos que vamos a evaluar nuestros modelos.

También podemos hacer una suposición lógica de la importancia de las columnas (Basandonos en la gravedad del suceso). Como por ejemplo que el sexo y la edad son variables de gran importancia ya que daban prioridad a mujeres y niños. 

También podemos suponer que la variable de clase "Pclass" debe ser importante, ya que en el barco iban personas de status social bastante alto. 

El nombre no debe ser de importancia para nuestros modelos.

Y la columna "Cabin" debe ser eliminada ya que cuenta con demasiados valroes Null.

**Estas suposiciones serán verificadas más adelante**

Veamos el comportamiento de la variable a predecir "Survived"

In [ ]:
titanicTrain.Survived.describe()

In [ ]:
titanicTrain['Survived'].value_counts()

El resultado obtenido nos indica que tan solo sobrevivieron 342 personas de las 891 que iban a bordo.

Veamoslo gráficamente:

(Para eso usaremos la librería de matplotlib asignada como plt)

In [ ]:
noSobrevivientes = titanicTrain.Survived[titanicTrain.Survived ==0].count()
sobrevivientes = titanicTrain.Survived[titanicTrain.Survived ==1].count()
plt.bar(['No sobrevivientes', 'Sobrevivientes'], [noSobrevivientes, sobrevivientes])
plt.title('Survived')

In [ ]:
plt.pie(titanicTrain.Survived.value_counts(),labels=['No sobrevivientes', 'Sobrevivientes'], autopct="%0.1f %%")
plt.title('Survived')

Ya sabemos cual fue el porcentaje de sobrevivientes. Ahora veamos cual es la cantidad de mujeres que sobrevivieron

In [ ]:
titanicTrain.Sex.head() #ver etiquetas

In [ ]:
titanicTrain.Survived[(titanicTrain.Survived == 1)&(titanicTrain.Sex.str.contains('female'))].count()

Esto nos arroja un resultado de 233, lo que significa que de los 342 sobrevivientes 233 son mujeres y el restante son hombres.

Ahora veamoslo gráficamente:

In [ ]:
plt.pie([233,109], labels=['female','male'],autopct="%0.1f %%" )
plt.title('Hombres y mujeres sobrevivientes')

Con este grafico podemos ver que el 68.1% de los sobrevivientes eran mujeres y el 31,9% restante eran hombres.

Lo mismo con el diagrama de barras:

In [ ]:
plt.bar(['female','male'], [233, 109], width=0.98)
plt.title('Sobrevivientes')

Una vez sabiendo cual es el porcentaje de mujeres y hombres sobrevivientes, queremos saber como se comporta la edad ante la columna "Survived". 
Para ello primero veamos qué edades tenemos en el conjunto de datos de entrenamiento.

In [ ]:
titanicTrain['Age'].describe()

Con esta descripción de la columna de edades podemos sacar conclusiones como que la edad minima es 0 años, la máxima es 80 y la edad media es de 29.

In [ ]:
plt.hist(titanicTrain['Age'])

Con el gráfico podemos interpretar que en el barco viajaba una mayor cantidad de niños que de adultos mayores, que la mayor cantidad de personas abordo estaban entre los 20 y 30 años, Entre otras cosas...

Observemos la cantidad de personas menores de 20 años que sobreviven y las que no:

In [ ]:
menores = titanicTrain[titanicTrain.Age < 20] #164 menores de 20 años
menores

In [ ]:
menores.Survived[menores.Survived == 1].count() #de 164 personas menores de 20, 79 sobrevivieron

In [ ]:
menores.Survived[menores.Survived == 0].count() #de 164 personas menores de 20 años, 85 no sobrevivieron.

Ahora obervemos esto mismo con las personas mayores de 60 años 

In [ ]:
mayores = titanicTrain[titanicTrain.Age >= 60] #26 personas tienen una edad igual o mayor a 60 años
mayores

In [ ]:
mayores.Survived[mayores.Survived == 1].count() #7 de las 26 personas mayores sobrevivieron

In [ ]:
mayores.Survived[mayores.Survived == 0].count() #19 de las 26 personas mayores no sobreviven

Y para finalizar el analisis de la columna "Age", observemos como resulta estos datos con las personas de edad media.

In [ ]:
media = titanicTrain[(titanicTrain.Age >= 20) & (titanicTrain.Age <= 30)] #245 personas tienen una edad entre 20 y 30 años
media

In [ ]:
media.Survived[media.Survived == 1].count() #87 de las 245 personas de edad media sobreviven

In [ ]:
media.Survived[media.Survived == 0].count() #158 de las 245 personas de edad media No sobreviven

Por último vamos a observar como se comporta la columna "Pclass" que es una de las columnas que hemos supuesto como importante ante la mortandad del accidente.

In [ ]:
titanicTrain['Pclass'].value_counts()

In [ ]:
plt.bar(['clase 3', 'clase 1', 'clase 2'],titanicTrain['Pclass'].value_counts())
plt.title('Distribución de clases en el barco')

Con este grafico podemos observar que la clase 3 es un poco más que el doble que la clase 1. Mientras que la clase 2 es un poco menor en tamaño que la clase 1.

Ahora observemos la mortandad en cada clase.

In [ ]:
#Observación de primera clase
classSobre = titanicTrain.Survived[(titanicTrain.Survived == 1)&(titanicTrain.Pclass == 1)].count()
classNoSobre = titanicTrain.Survived[(titanicTrain.Survived == 0)&(titanicTrain.Pclass == 1)].count()
plt.bar(['No sobrevivientes', 'Sobrevivientes'], [classNoSobre,classSobre])
plt.title('Distribución de muerte en primera clase')

In [ ]:
plt.pie([classSobre, classNoSobre], labels=['Sobrevivientes', 'No sobrevivientes'],autopct="%0.1f %%" )
plt.title('Distribuación de muerte en primera clase')

In [ ]:
#Observación de segunda clase
classSobre = titanicTrain.Survived[(titanicTrain.Survived == 1)&(titanicTrain.Pclass == 2)].count()
classNoSobre = titanicTrain.Survived[(titanicTrain.Survived == 0)&(titanicTrain.Pclass == 2)].count()
plt.bar(['No sobrevivientes', 'Sobrevivientes'], [classNoSobre,classSobre])
plt.title('Distribuación de muerte en segunda clase')

In [ ]:
plt.pie([classSobre, classNoSobre], labels=['Sobrevivientes', 'No sobrevivientes'],autopct="%0.1f %%" )
plt.title('Distribuación de muerte en segunda clase')

In [ ]:
#Observación de tercera clase
classSobre = titanicTrain.Survived[(titanicTrain.Survived == 1)&(titanicTrain.Pclass == 3)].count()
classNoSobre = titanicTrain.Survived[(titanicTrain.Survived == 0)&(titanicTrain.Pclass == 3)].count()
plt.bar(['No sobrevivientes', 'Sobrevivientes'], [ classNoSobre, classSobre])
plt.title('Distribuación de muerte en tercera clase')

In [ ]:
plt.pie([classSobre, classNoSobre], labels=['Sobrevivientes', 'No sobrevivientes'],autopct="%0.1f %%" )
plt.title('Distribuación de muerte en tercera clase')

Como podemos observar en las gráficas y como se podía tener una idea, la primera clase tuvo una gran cantidad de sobrevivientes (63%) en comparación con las demás clases. Mientras que en la 3ra clase solo sobrevivió un 24.2% de la tripulación, teniendo en cuenta que la cantidad de personas pertenecientes a la 3ra clase era más del doble que los de la primera clase.

Una vez habiendo analizado las columnas que creímos importantes, vamos a observar las columnas restantes. 

In [ ]:
sns.pairplot(titanicTrain)

Con este grafico podemos observar las variables numericas que nos hace falta por explorar. Observamos que la columna "sibps" que representa el numero de hermanos o conyugues a bordo; tiene en su mayoria numero bajos, lo que quiere decir que existían pocas personas con este tipo de parentesco. 

De igual manera se observa con la columna "Parch" que representa el numero de padres e hijos a bordo; tiene en su mayoria numeros bajos. 

y por último la columna "Fare" que representa la tarifa de pasajero. Esta columna tiene numeros muy bajos, lo que tiene sentido porque la gran mayoría de la tripulación eran de la 3ra clase. Quizás esta columna tenga importacia ya que dependiendo la tarifa se puede suponer la clase o estrato de la persona. Pero es algo que veremos en un grafico más adelante.

Falta por observar las caolumnas etiquetadas como Cabin, Embarked y Ticket. Observemos:

In [ ]:
titanicTrain[['Cabin','Ticket','Embarked']]

In [ ]:
#Observemos la columna Cabin
titanicTrain['Cabin'].value_counts()

In [ ]:
titanicTrain['Cabin'].isnull().sum()

De 891 registros, la columna Cabin contiene 687 valores nulos, lo que significa que debemos eliminarla por su gran cantidad de datos vacios. 

In [ ]:
titanicTrain.drop('Cabin', axis = 1, inplace=True)
titanicTest.drop('Cabin', axis = 1, inplace=True)

In [ ]:
#Observemos la columna Ticket
titanicTrain['Ticket'].value_counts()

In [ ]:
#Observemos la columna Cabin
titanicTrain['Ticket'].isnull().sum()

Esta columna tiene sus registros completos, pero no creo que por el ticket de una persona se pueda decidir si alguien sobrevive o no sobrevive. Sin embargo, se podrá ver la relación que tiene esta columna con nuestro Target (Objetivo de predecir supervivencia o no) en un grafico más adelante. 

In [ ]:
#veamos la columna Embarked
titanicTrain['Embarked'].isnull().sum()

In [ ]:
titanicTrain['Embarked'].value_counts()

Como vemos, la columna "Embarked" tiene dos registros faltantes que no representan ningún problema. Se pueden eliminar estas dos filas o llenarlas con el promedio de la embarcación. Tambien vemos que existen 3 embarcaciones: S,C,Q y con base a esto nos podemos preguntar ¿La embarcación representó un factor para la supervivencia de los tripulantes ? ¿Como se presenta la mortandad en cada una de las embarcaciones? Vamos a responder estas preguntas.

In [ ]:
#Embarked S
sobrevivientes = titanicTrain.Embarked[(titanicTrain.Survived == 1) & (titanicTrain.Embarked.str.contains('S'))].count()
noSobrevivientes = titanicTrain.Embarked[(titanicTrain.Survived == 0) & (titanicTrain.Embarked.str.contains('S'))].count()
plt.pie([sobrevivientes, noSobrevivientes],labels=['Sobrevivientes', 'No Sobrevivientes'],autopct="%0.1f %%")
plt.title('Mortalidad de la embarcación S')

In [ ]:
#Embarked C
sobrevivientes = titanicTrain.Embarked[(titanicTrain.Survived == 1) & (titanicTrain.Embarked.str.contains('C'))].count()
noSobrevivientes = titanicTrain.Embarked[(titanicTrain.Survived == 0) & (titanicTrain.Embarked.str.contains('C'))].count()
plt.pie([sobrevivientes, noSobrevivientes],labels=['Sobrevivientes', 'No Sobrevivientes'],autopct="%0.1f %%")
plt.title('Mortalidad de la embarcación C')

In [ ]:
#Embarked Q
sobrevivientes = titanicTrain.Embarked[(titanicTrain.Survived == 1) & (titanicTrain.Embarked.str.contains('Q'))].count()
noSobrevivientes = titanicTrain.Embarked[(titanicTrain.Survived == 0) & (titanicTrain.Embarked.str.contains('Q'))].count()
plt.pie([sobrevivientes, noSobrevivientes],labels=['Sobrevivientes', 'No Sobrevivientes'],autopct="%0.1f %%")
plt.title('Mortalidad de la embarcación Q')

Con estas gráficas podemos darnos una idea de la mortaldiad correspondiente a cada embarcación. A simple vista podemos ver que la mortalidad en cada una de ellas se da de manera común, por lo tanto no hay indicio de que pertenecer a cierta embarcación tuviera conseciencia en sobrevivir o no al hundimiento del barco. Pero no podemos descartarla aún. 

En cuanto a las columnas de "PassengerId", "Name" y "Ticket" las descartaremos ya que estas columnas de tipo Object dificilmente se pueden pasar a tipo numerico, y además no podrían aportar nada a nuestros modelos.

In [ ]:
titanicTrain.drop(['PassengerId','Name', "Ticket"], axis = 1, inplace = True)
titanicTest.drop(['PassengerId','Name', "Ticket"], axis = 1, inplace = True)

In [ ]:
print(titanicTrain)
print(titanicTest)

Una vez hemos visto el comportamiento de cada una de las columnas de nuestros datos; vamos a observar la correlación de ellas con nuestro target "Survived". Pero como esta correlación solo se puede dar con variables de tipo numerico, debemos transformar las variables "Sex" y "Embarked" que son categoricas a variables numericas.

In [ ]:
#conjunto de test
titanicTrain['Sex'].replace(['male','female'],[0,1], inplace = True)
titanicTrain['Embarked'].replace(['S','C','Q'],[1,2,3], inplace = True)

#conjunto de test
titanicTest['Sex'].replace(['male','female'],[0,1], inplace = True)
titanicTest['Embarked'].replace(['S','C','Q'],[1,2,3], inplace = True)

In [ ]:
titanicTrain

In [ ]:
titanicTest

Ahora si podemos usar nuestro cuadro de correlación gracias a la libreria seaborn que hemos implementado al inicio de nuestro código.

In [ ]:
corr = titanicTrain.corr()
sns.heatmap(corr, cmap='RdBu', annot=True, fmt=".2f")

y acá podemos ver los porcentajes ordenados de manera descendente:

In [ ]:
corr[['Survived']].sort_values(by = 'Survived',ascending = False)

Como podemos observar, los coeficientes de correlación con respecto a nuestro target son demasiado bajos, pero debemos trabajar con ellos. La puntuación más alta es la de "Sex" que corresponde a un 54% de correlación, por tal motivo necesitaremos esa columna para entrenar nuestros algoritmos.

La columna "Fare" es la puntuación más alta por debajo de la columna "Sex" con un 25% de correlación. Pero si nos situamos en la intersección entre la columna "Fare" y la columna "Pclass" podemos ver que entre estas dos variables existe una relación negativa de un 55%,lo que significa que en esa medida las dos columnas representan lo mismo y una de las dos es innecesaria. Por tal motivo vamos a eliminar la columna "Fare" y nos quedaremos con la columna "Plcass" ya que tiene una mayor relación con nuestra columna "Survived". *(Las relaciones son independiente del signo, por tal motivo puede existir un porcentaje de correlación con ambos signos y representar el mismo nivel de relación)*

Seguido de esto usaremos la columna "Embarked" que tiene un 10% de relación.  

Por último eliminaremos las columnas "Parch" y "SipSp" ya que haciendo un análisis mental (sin soporte estadistico) considero que tener lazos familiares no aporta en gran medida a la supervivencia de una persona. Sin embargo, NO eliminaremos la columna de "Age" a pesar de tener una correlación tan baja, ya que se debió tener mayor prioridad en salvar a niños que a personas adultas.

De tal manera nuestros conjuntos de datos  quedan de la siguiente manera: 

In [ ]:
titanicTrain = titanicTrain.loc[:,['Sex','Pclass','Age','Embarked','Survived']]
titanicTest = titanicTest.loc[:,['Sex','Pclass','Age','Embarked']]
titanicTrain, titanicTest

Una vez hecho nuestro análisis y seleccionado nuestras características, estamos listos para seguir con nuestra siguiente etapa de preprocesamiento de datos.

# 3. Preparar los datos

En esta etapa se suele hacer el siguiente preprocesamiento: 
1. Transformar variables categóricas a numéricas.
2. Tratar los datos faltantes.
3. Tratar los datos atípicos.
4. Considerar el escalado de los datos.


1. Para nuestro primer punto no debemos hacer ninguna transformación ya que en el proceso de análisis fuimos tranformando las columnas necesarias para nuestra observación y ahora todo nuestro conjunto de datos es de tipo numérico.
2. **Tratar los datos faltantes**. En este punto si debemos analizar cada columna para ver la cantidad de datos que faltan en cada una de ellas y como tratarlos.

In [ ]:
print(titanicTrain['Sex'].isnull().sum()) #Cantidad de valores vacíos en entrenamiento
print(titanicTest['Sex'].isnull().sum())#Cantidad de valores vacíos en test

In [ ]:
print(titanicTrain['Pclass'].isnull().sum()) #Cantidad de valores vacíos en entrenamiento
print(titanicTest['Pclass'].isnull().sum())#Cantidad de valores vacíos en test

In [ ]:
print(titanicTrain['Age'].isnull().sum()) #Cantidad de valores vacíos en entrenamiento 
print(titanicTest['Age'].isnull().sum()) #Cantidad de valores vacíos en test

In [ ]:
print(titanicTrain['Embarked'].isnull().sum()) #Cantidad de valores vacíos en entrenamiento 
print(titanicTest['Embarked'].isnull().sum()) #Cantidad de valores vacíos en test

In [ ]:
print(titanicTrain['Survived'].isnull().sum()) #Cantidad de valores vacíos en entrenamiento

Teniendo en cuenta que las columnas "Age" y "Embarked" tiene valores vacíos vamos a tratarlos.

La columna "Age" tiene 177 valores faltantes de 891. Existen distintas formas de rellenar estos valore faltantes, pero en esta ocación los llenaremos con el valor promedio de edades.

In [ ]:
promedio = titanicTrain['Age'].mean()
titanicTrain['Age'].fillna(promedio, inplace = True)
print(titanicTrain['Age'].isnull().sum())

#Ahora con el conjunto de prueba
promedio = titanicTest['Age'].mean()
titanicTest['Age'].fillna(promedio, inplace = True)
print(titanicTest['Age'].isnull().sum())

De esta manera hemos rellenado los valores vacíos de la columna "Age" con el promedio de edad. Ahora nos falta la columna "Embarked" el cual tiene 2 registros faltantes de 891. En este caso vamos a rellenar estos dos valores vacíos con el numero que más se repite.

In [ ]:
titanicTrain['Embarked'].value_counts()

In [ ]:
titanicTrain['Embarked'].fillna(1, inplace = True)
print(titanicTrain['Embarked'].isnull().sum())

In [ ]:
print(titanicTrain.info())
print(titanicTest.info())

Ahora ya no tenemos valores vacíos en nuestro conjunto de datos, hemos terminado el segundo punto y podemos seguir con nuestro preprocesamiento de datos.

3.  **Tratar los datos atípicos.**
Para este punto solo debemos centrarnos en la columna "Age" ya que las demás columnas son categóricas y no tienen mayor variedad de números, de esta manera no existe riesgo de valores Outliers.

In [ ]:
sns.boxplot(titanicTrain['Age'])

Esta gráfica nos muestra algunos puntos de ánalisis, el cuartil inferior que es un poco mayor a 20, la media que es 30 y el cuartil superior que está al rededor de 35. los bigotes (lineas verticales de los lados) exponen que todo numero que esté dentro de estos limites se consideran normales, mientras que los que están por fuera se consideran datos atípicos. En este caso se consideran datos atípicos las edades inferiores a 3 y superiores a 53 aproximadamente. Dependiendo del caso se deberían eliminar estos datos para que no afecten el rendimiento de nuestros agoritmos, pero en esta ocación no es necesario, ya que sí es posible que una persona de 70 u 80 años estuviera en el bote, o un bebé de apenas 1 año de edad. Caso contrario sería de una persona de 150 o 200 años, o alguien con -3 años, en ese momento si se deberían eliminar dichos registros ya que es ilógico que un caso así se presente.  

Una vez considerado los datos Outliers podemos seguir con nuestro último paso de escalado de datos.

4. **Considerar el escalado de los datos**. El escalado de los datos se debe realizar cuando existen columnas con diferencias de valores muy altos, como por ejemplo el valor de un apartamento (200000000) y el numero de habitaciones de este (4). El escalado se hace precisamente para reducir esta diferencia y poder dar los mejores resultados con nuestros algoritmos. Como nuestro problema es de clasificación y no presentamos diferencias en nuestros valores de columnas, entonces no es necesario hacer el escalado de los datos. 

Con esto último hemos terminado nuestro proceso de pre-procesamiento y ya tenemos listos nuestros datos para poder entrenar nuestros modelos de la mejor manera. 

In [ ]:
titanicTrain

In [ ]:
titanicTest

# 4. Hacer predicciones

En esta etapa entrenaremos nuestros modelos con la data previamente procesada y haremos nuestras predicciones. Utilizaremos modelos de clasificación tales como :

* Logistic Regression
* KNN o k-Nearest Neighbors
* SVM o Support Vector Machine
* Naive Bayes classifier
* Decision Tree
* Random Forrest

Compararemos y mejoraremos los resultados.

In [ ]:
y_train = titanicTrain.loc[:, 'Survived']
x_train = titanicTrain.drop('Survived', axis = 1)
x_test = titanicTest

In [ ]:
#Regresión logistica
from sklearn.linear_model import LogisticRegression #Importamos la librería del modelo

clasificador = LogisticRegression(random_state = 0)
clasificador.fit(x_train, y_train) #Entrenamos el modelo con los datos de entrenamiento 

y_pred = clasificador.predict(x_test) #Hacemos predicciones sobre el conjunto de test

cv = cross_val_score(estimator = clasificador, X = x_train, y = y_train, cv = 10) # metodo para obtener la precisión (Validación cruzada)
cvRegression = cv.mean()
stdRegression = cv.std()
print("promedio Validación cruzada Regresión logística: ", cvRegression)
print('Varianza de Regresión logística: ',stdRegression)

In [ ]:
#KNN o k-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier #Importamos la librería del modelo

clasificador = KNeighborsClassifier()
clasificador.fit(x_train, y_train) #Entrenamos el modelo con los datos de entrenamiento 

y_pred = clasificador.predict(x_test) #Hacemos predicciones sobre el conjunto de test

cv = cross_val_score(estimator = clasificador, X = x_train, y = y_train, cv = 10) #metodo para obtener la precisión
cvKnn = cv.mean()
stdKnn = cv.std()
print("promedio Validación cruzada KNN: ", cvKnn)
print('Varianza de KNN: ',stdKnn)

In [ ]:
#SVM o Support Vector Machine
from sklearn.svm import SVC #Importamos la librería del modelo
#Para implementar el modelo de SVM es obligatorio tener los datos escalados
from sklearn.preprocessing import StandardScaler #Libreria para escalar los datos

scaler = StandardScaler()
x_entrenamiento = scaler.fit_transform(x_train) #Escalar los datos de entrenamiento
x_prueba = scaler.fit_transform(x_test)

clasificador = SVC(random_state = 0) 
clasificador.fit(x_entrenamiento, y_train) #Entrenamos el modelo con los datos de entrenamiento 

y_pred = clasificador.predict(x_prueba) #Hacemos predicciones sobre el conjunto de test

cv = cross_val_score(estimator = clasificador, X = x_train, y = y_train, cv = 10) #metodo para obtener la precisión
cvSVM = cv.mean()
stdSVM = cv.std()
print("promedio Validación cruzada SVM: ", cvSVM)
print('Varianza de SVM: ',stdSVM)

In [ ]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB #Importamos la librería del modelo

clasificador = GaussianNB()
clasificador.fit(x_train, y_train) #Entrenamos el modelo con los datos de entrenamiento 

y_pred = clasificador.predict(x_test) #Hacemos predicciones sobre el conjunto de test

cv = cross_val_score(estimator = clasificador, X = x_train, y = y_train, cv = 10) #metodo para obtener la precisión
cvNB = cv.mean()
stdNB = cv.std()
print("promedio Validación cruzada NB: ", cvNB)
print('Varianza de NB: ',stdNB)

In [ ]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier #Importamos la librería del modelo

clasificadorTree = DecisionTreeClassifier(random_state = 0)
clasificadorTree.fit(x_train, y_train) #Entrenamos el modelo con los datos de entrenamiento 

y_pred = clasificadorTree.predict(x_test) #Hacemos predicciones sobre el conjunto de test

cv = cross_val_score(estimator = clasificadorTree, X = x_train, y = y_train, cv = 10) #metodo para obtener la precisión
cvTree = cv.mean()
stdTree = cv.std()
print("promedio Validación cruzada Decision Tree: ", cvTree)
print('Varianza de Decision Tree: ',stdTree)

In [ ]:
#Random Forest 
from sklearn.ensemble import RandomForestClassifier #Importamos la librería del modelo

clasificadorForest = RandomForestClassifier(random_state = 0)
clasificadorForest.fit(x_train, y_train) #Entrenamos el modelo con los datos de entrenamiento 

y_pred = clasificadorForest.predict(x_test) #Hacemos predicciones sobre el conjunto de test

cv = cross_val_score(estimator = clasificadorForest, X = x_train, y = y_train, cv = 10) #metodo para obtener la precisión
cvForest = cv.mean()
stdForest = cv.std()
print("promedio Validación cruzada Random Forest: ", cvForest)
print('Varianza de Random Forest: ',stdForest)

In [ ]:
#score = [scoreRegression, scoreKnn, scoreSVM, scoreNB, scoreTree, scoreForest]
meanValidaCruz = [cvRegression, cvKnn, cvSVM, cvNB, cvTree, cvForest]
stdValidaCruz = [stdRegression, stdKnn, stdSVM, stdNB, stdTree, stdForest]
labels = ['Regresion lineal', 'Knn', 'SVM', 'Naive Bayes', 'Árbol de decision', 'Random Forest']

scoreModels = pd.DataFrame({'Models': labels,
                            'Vali. Cruz.': meanValidaCruz,
                            'Varianza': stdValidaCruz})
scoreModels.sort_values(by = 'Vali. Cruz.', ascending = False)

In [ ]:
width = 0.30
ind =np.arange(6)
plt.bar(ind ,meanValidaCruz, width, label='VC')
plt.bar(ind + width,stdValidaCruz, width, label='Varianza')
#plt.bar(ind + width*2,score, width, color='r', label='Score')
plt.xticks(ind + width/2,('Regression', 'Knn', 'SVM', 'NB', 'Tree','Forest'))
plt.legend()
plt.title("Puntuaciones")

Ahora que sabemos la precisión de cada modelo debemos mejorar las puntuaciones obtenidas. Como los modelos Random Forest y Árbol de decisión presentaron los mejores resultados; nos enfocaremos en optimizar dichos modelos. Para esto haremos uso de la librería GridSearchCV de Sklearn que nos permite hallar los mejores parametros para implementar nuestros algoritmos. 

# 5. Mejorar los resultados

In [ ]:
#GirdSearchCV con el modelo de árbol de decisión

from sklearn.model_selection import GridSearchCV #Importamos la librería

minSplit = np.arange(20,26,1)
maxDepth = np.arange(4.0,4.6,0.1)

parametros = [{'criterion': ['gini'],'max_depth': [3.5,4.1], 'min_samples_split': minSplit},
              {'criterion': ['entropy'], 'max_depth':  [3.5,4.1], 'min_samples_split':  minSplit}
             ] #Definimos los parametros para que la librería nos indique cuál es el mejor

gridSearch = GridSearchCV(estimator = clasificadorTree, 
                          param_grid = parametros,
                          scoring = 'accuracy',
                          cv = 10) #En el constructor pasamos el modelo que queremos mejorar, los parametros, medida de calificacion y nuemero de pruebas.

gridSearch = gridSearch.fit(x_train, y_train) #Entrenamos el GridSearchCV
print(gridSearch.best_score_) #vemos como la puntuación cv pasó de 0.7924 a  0.8215
print(gridSearch.best_params_) #Muestras los parametros más optimos


#Aplicacion de parametros
clasificadorTree = DecisionTreeClassifier(criterion= 'gini', 
                                          max_depth= 4.1,min_samples_split = 21,
                                          random_state = 0)

clasificadorTree.fit(x_train, y_train) #Entrenamos el modelo con los datos de entrenamiento 

y_pred = clasificadorTree.predict(x_test) #Hacemos predicciones sobre el conjunto de test

scoreTreeGS = clasificadorTree.score(x_train, y_train) #Obtenemos la precisión del algoritmo
print('Precisión de algoritmo de Decision Tree: ',scoreTreeGS)

cv = cross_val_score(estimator = clasificadorTree, X = x_train, y = y_train, cv = 10) #Metodo para obtener la precisión
cvTreeGS = cv.mean()
stdTreeGS = cv.std()
print("promedio Validación cruzada Decision Tree: ", cvTreeGS)
print('Varianza de Decision Tree: ',stdTreeGS)



In [ ]:
#Cuadro comparativo de resultados
defectpVsGSTree = pd.DataFrame({'Parametros. Árbol de Decisión':['Por Defecto', 'GridSearchCV'],
                                'Mean Vali. Cruz.':[cvTree,cvTreeGS ],
                                'Varianza': [stdTree, stdTreeGS]})

defectpVsGSTree

In [ ]:
#GirdSearchCV con el modelo de bosques aleatorios

nEstimators = np.arange(80,86)
maxDepth = np.arange(4.0,4.3,0.1)

#best'max_features': ['auto'],
#best'criterion' :['entropy']
parametros = { 
    'n_estimators': nEstimators,
    'max_depth' :  maxDepth,
}#Definimos los parametros que para que la librería nos indique cuál es el mejor

gridSearch = GridSearchCV(estimator = clasificadorForest, 
                          param_grid = parametros,
                          scoring = 'accuracy',
                          cv = 10)#En el constructor pasamos el modelo que queremos mejorar, los parametros, medida de calificacion y nuemero de pruebas.

gridSearch = gridSearch.fit(x_train, y_train) #Entrenamos el GridSearch
print(gridSearch.best_score_) #vemos como la puntuación de cv pasó de 0.8025 a 0.8237 
print(gridSearch.best_params_) #Vemos los parametros más optimos


#Aplicación de parametros 
clasificadorForest = RandomForestClassifier(max_features ='auto',
                                            criterion = 'entropy',
                                            n_estimators = 84,
                                            max_depth =  4,
                                            random_state = 0)
clasificadorForest.fit(x_train, y_train) #Entrenamos el modelo con los datos de entrenamiento 

y_pred = clasificadorForest.predict(x_test) #Hacemos predicciones sobre el conjunto de test

scoreForestGS = clasificadorForest.score(x_train, y_train) #Obtenemos la precisión del algoritmo
print('Precisión de algoritmo de Random Forest: ',scoreForestGS)

cv = cross_val_score(estimator = clasificadorForest, X = x_train, y = y_train, cv = 10) #Otro metodo para obtener la precisión
cvForestGS = cv.mean()
stdForestGS = cv.std()
print("promedio Validación cruzada Random Forest: ", cvForestGS)
print('Varianza de Random Forest: ',stdForestGS)

In [ ]:
#Cuadro comparativo de resultados
defectpVsGSForest = pd.DataFrame({'Bosques Aleatorios':['Por Defecto', 'GridSearchCV'],
                                'Mean Vali. Cruz.':[cvForest,cvForestGS],
                                'Varianza': [stdForest, stdForestGS]})

defectpVsGSForest

In [ ]:
#Cuadro comparativo de modelos
modelos = pd.DataFrame({'Modelo': ['DecisionTree', 'RandomForest'],
                      'Precision': [cvTreeGS, cvForestGS],
                       'Varianza': [stdTreeGS, stdForestGS]
                      })
modelos.sort_values(by='Precision',  ascending = False)


Con estos resultados hemos finalizado el proceso de predicción de la supervivencia de los tripulantes del titanic. 
En este punto hemos hecho un análisis, pre-procesamiento de datos, selección, entrenamiento y optimización de modelos para la solución del problema planteado. 

Hemos terminado con dos modelos que presentan los mejores resultados en comparación con los demás escogidos. Estos son: El modelo de RandomForest con una precisión del 82,26% y con una varianza del 3,5%. El otro modelo es el de DecisionTree, que presenta una precisión de 82,21% con una varianza del 3,8%

Con estas conclusiones pueden surgir preguntas como: 
1. ¿Pueden mejorar estos resultados ?
2. ¿Como podemos mejorar estos resultados ? 
3. ¿Hemos hecho una correcta selección de caracteristicas ? 
4. ¿Hemos hecho un correcto pre-procesamiento de datos ? 
5. ¿Podríamos implementar modelos que se ajusten mejor a este problema ? 

Si tiene alguna respuesta a estas preguntas podría dejarlas en un comentario, o si tiene dudas o sugerencias podría hacermelo saber para crear retroalimentación y aprender entre nosotros. 
